In [2]:
from h3 import h3

import pandas as pd

from shapely.geometry import Point,Polygon

from shapely.geometry import shape

import folium

# from cartoframes.viz import Map, Layer, color_continuous_style, palettes, histogram_widget

import numpy as np
import geopandas as gpd

# from cartoframes.viz import color_category_style, color_bins_style, popup_element, size_bins_style

# from cartoframes.viz import  basic_widget , category_widget

from shapely.geometry import shape, GeometryCollection

import time

import seaborn as sn

from sklearn.cluster import DBSCAN, KMeans

from matplotlib import pyplot as plt

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Содержание:
* [Грузим датасет лавки](#zero-bullet)
* [Грузим и обрабатываем датасет самоката](#first-bullet)
* [Сцепляем датасет самоката и лавки](#first-half-bullet)

In [40]:
df_lavka = gpd.read_file('гексы_лавки_изохроны_охват.geojson', driver='GeoJSON')

In [41]:
df_lavka.head()

,external_id,hex_gid,C1C2_res_500,C1C2_res_500_no_borders,C1C2_res_1000,C1C2_res_1000_no_borders,rank_within_zone,lavka_id,lavka_slug,hex_coverage_rank_percent,...,lat,long,C1C2_res_6m_no_borders,C1C2_res_6m,C1C2_res_12m_no_borders,C1C2_res_12m,C1C2_res,lavka_exist_month,count_orders_sum,geometry
0,202983,891181b4cdbffff,842.0,1577.0,3357.0,5450.0,44.0,NaN,None,9.0,...,37.730615,55.593054,125.0,1.0,3424.0,1775.0,1.0,8,0.0,"POLYGON ((37.72786 55.59230, 37.73024 55.59124..."
1,375098,891181b4ccfffff,181.0,1527.0,927.0,6644.0,31.0,NaN,None,8.0,...,37.738874,55.595324,771.0,379.0,11977.0,2506.0,11.0,1,0.0,"POLYGON ((37.73612 55.59457, 37.73850 55.59351..."
2,202983,891181b4c1bffff,1725.0,1794.0,6059.0,7705.0,34.0,NaN,None,8.0,...,37.742000,55.597897,3745.0,3564.0,17657.0,9451.0,4.0,8,0.0,"POLYGON ((37.73925 55.59714, 37.74163 55.59608..."
3,202983,891181b4c0fffff,1591.0,1883.0,5260.0,6399.0,43.0,NaN,None,8.0,...,37.750262,55.600167,457.0,391.0,10339.0,7644.0,0.0,8,0.0,"POLYGON ((37.74751 55.59941, 37.74989 55.59835..."
4,202983,891181b4d5bffff,1313.0,1348.0,2367.0,3883.0,42.0,NaN,None,9.0,...,37.764783,55.607583,1245.0,1150.0,8378.0,4425.0,1.0,8,0.0,"POLYGON ((37.76203 55.60683, 37.76441 55.60577..."


In [4]:
gdf_lavki_loc = gpd.read_file('gdf_lavki_loc.geojson', driver='GeoJSON')

In [6]:
def get_points(lat, long):
    return[ Point (j,i) for i, j in zip(lat, long)]

def get_gdf(df, geometry, crs):
    return gpd.GeoDataFrame(df, geometry=geometry, crs='epsg:%s'%str(crs))

In [7]:
df_samokat=pd.read_csv('yt__home_maps_geocoder_geocoding_result_lavka_samokat6209c75da785b0d4.csv', sep=',')

df_samokat.head()

df_samokat.drop(0,inplace=True)

df_samokat['point_lat'] = df_samokat['point_lat'].astype(float)
df_samokat['point_lon'] = df_samokat['point_lon'].astype(float)

df_samokat['geometry']=get_points(df_samokat['point_lat'].astype(float),df_samokat['point_lon'].astype(float))

gdf_samokat=get_gdf(df_samokat, crs=4326, geometry='geometry')

gdf_samokat_mos = gdf_samokat[(gdf_samokat['point_lon']>37.0)&(gdf_samokat['point_lon']<39.0)&(gdf_samokat['point_lat']>50.0)]



gdf_samokat_mos.drop_duplicates(inplace=True)

import yt.wrapper as yt

yt.config["proxy"]["url"] = "hahn"

from yt.wrapper import YtClient
client = YtClient(proxy="hahn", config={"tabular_data_format": "dsv"})


table = "//home/taxi-ci/samokat/v2/coverage_actual"


# yt.run_sort(table, sort_by=["live_agglomeration_node_id"])

#%%time 

df=pd.DataFrame()

for n,row in enumerate(yt.read_table(yt.TablePath(table),format="yson")):
    df_tmp=pd.DataFrame({n:row}).T
    df=pd.concat([df,df_tmp])


#     print (df_tmp)

df_samokat = df.merge(gdf_samokat_mos, left_on='address', right_on='input_request', how='left')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
df_samokat.isnull().mean()

acceptingOrders           0.000000
address_x                 0.000000
coverage_id               0.000000
full_response_coverage    0.000000
full_response_list        0.000000
full_response_settings    0.000000
lavka_ids                 0.000000
minimalAmount             0.000000
showcaseId                0.000000
type                      0.000000
utc_dt                    0.000000
utc_tm                    0.000000
zone                      0.000000
input_request             0.539936
input_lang                0.539936
input_lon                 0.539936
input_lat                 0.539936
distance                  0.539936
precision                 0.539936
accuracy                  0.539936
point_lon                 0.539936
point_lat                 0.539936
geoid                     0.539936
address_y                 0.539936
kind                      0.539936
geometry                  0.539936
dtype: float64

In [13]:
df_samokat_msk = df_samokat[df_samokat.zone=='msk']

In [14]:
df_samokat_msk.isnull().mean()

acceptingOrders           0.000000
address_x                 0.000000
coverage_id               0.000000
full_response_coverage    0.000000
full_response_list        0.000000
full_response_settings    0.000000
lavka_ids                 0.000000
minimalAmount             0.000000
showcaseId                0.000000
type                      0.000000
utc_dt                    0.000000
utc_tm                    0.000000
zone                      0.000000
input_request             0.106291
input_lang                0.106291
input_lon                 0.106291
input_lat                 0.106291
distance                  0.106291
precision                 0.106291
accuracy                  0.106291
point_lon                 0.106291
point_lat                 0.106291
geoid                     0.106291
address_y                 0.106291
kind                      0.106291
geometry                  0.106291
dtype: float64

In [15]:
df_samokat_msk['geometry_zone'] = df_samokat_msk['full_response_coverage'].apply(lambda x: None if pd.isnull(x) else Polygon(x['geometry']['coordinates'][0]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
df_samokat['geometry_zone']

0      POLYGON ((37.406051 55.726402, 37.406147 55.72...
1      POLYGON ((37.406051 55.726402, 37.406147 55.72...
2      POLYGON ((37.658879 55.776149, 37.656771 55.77...
3      POLYGON ((37.658879 55.776149, 37.656771 55.77...
4      POLYGON ((37.832346 55.639171, 37.826207 55.63...
                             ...                        
934    POLYGON ((56.2777 58.117199, 56.281002 58.1175...
935    POLYGON ((56.199017 57.991301, 56.201813 57.98...
936    POLYGON ((56.224767 57.998374, 56.224318 57.99...
937    POLYGON ((56.290596 57.987862, 56.29059 57.989...
938    POLYGON ((56.204946 57.993132, 56.202646 57.99...
Name: geometry_zone, Length: 939, dtype: object

In [16]:
gdf_samokat_msk = get_gdf(df_samokat_msk[pd.isnull(df_samokat_msk['kind'])],'geometry_zone', 4326 )

In [150]:
df_samokat_msk.set_geometry('geometry')[['geometry','address_x']]

,geometry,address_x
0,POINT (37.42911 55.72536),Красных зорь 59Г
1,POINT (37.42911 55.72536),Красных зорь 59Г
2,POINT (37.67822 55.76674),Денисовский 30стр10
3,POINT (37.67822 55.76674),Денисовский 30стр10
4,POINT (37.85014 55.63589),Угрешская 15а
...,...,...
456,POINT (37.53706 55.77296),3-й Хорошёвский 5стр1
457,POINT (37.53706 55.77296),3-й Хорошёвский 5стр1
458,POINT (37.70990 55.73714),Подъемный 1
459,POINT (37.70990 55.73714),Подъемный 1


In [22]:
gdf_samokat_msk.loc[25,'geometry']=Point( 37.931255, 55.970131)

In [76]:
gdf_samokat_msk.loc[gdf_samokat_msk['address_x']=='Жуковского 7','geometry']=Point( 37.324569, 55.815564)

In [75]:
gdf_samokat_msk.loc[gdf_samokat_msk['address_x']=='Жуковского 7','geometry']

106    POINT (55.81556 37.32457)
Name: geometry, dtype: geometry

In [79]:
gdf_samokat_msk.loc[79,'geometry'] = Point( 37.696516, 55.816713)

In [80]:
gdf_samokat_msk[pd.isnull(gdf_samokat_msk['geometry'])]

,acceptingOrders,address_x,coverage_id,full_response_coverage,full_response_list,full_response_settings,lavka_ids,minimalAmount,showcaseId,type,...,distance,precision,accuracy,point_lon,point_lat,geoid,address_y,kind,geometry,geometry_zone


In [17]:
dic_point_samokat ={26: Point(55.922449, 37.832324),
43: Point(55.886786, 37.443173),
44: Point(55.886786, 37.443173),
54:  Point( 55.902717, 37.40459),
64: Point(55.802163, 37.465461),
65: Point(55.802163, 37.465461),
78: Point(55.816713, 37.696516),
79: Point(55.816713, 37.696516),
95: Point(55.754162, 37.880958),
96:  Point( 55.754162, 37.880958),
329: Point(55.820217, 37.329546), 
330: Point(55.820217, 37.329546),
106: Point(55.815564, 37.324569),
107: Point(55.753519, 37.862660),
108: Point(55.753519, 37.862660),
118: Point(55.510558, 37.564168),
135:Point(55.796987, 37.984849),
161:Point(55.823550, 37.933950),
162: Point(55.721518, 37.452749), 
163:Point( 55.721518, 37.452749),
182:  Point(55.889477, 37.479600),
183: Point(55.889477, 37.479600),
193: Point(55.695936, 37.813764),
194: Point(55.695936, 37.813764),
210: Point(55.744875, 37.553199),
211: Point(55.744875, 37.553199),
212: Point(55.744875, 37.553199),
254: Point(55.894535, 37.716046),
277: Point(55.908580, 37.868256),
280: Point(55.579210, 38.219551),
286: Point(55.586257, 37.674247),
287: Point(55.586257, 37.674247),
298: Point(55.704681, 37.323087),
299: Point(55.704681, 37.323087),
302: Point(55.630533, 37.798691),
307: Point(55.605882, 38.075282),
348: Point(55.919685, 37.746373),
368: Point(55.797534, 37.947344),
371: Point(55.670935, 37.286426),
372: Point(55.670935, 37.286426),
403: Point(55.815666, 37.635206),
404: Point(55.815666, 37.635206),
420: Point(55.793804, 37.818741),
421: Point(55.793804, 37.818741),
422: Point(55.687863, 37.898278),
444: Point(55.817411, 37.593147),
445: Point(55.817411, 37.593147),
455: Point(55.698944, 37.919604),
 }

for k,v in dic_point_samokat.items():
    gdf_samokat_msk.loc[k,'geometry']=v

gdf_samokat_msk['geometry']=gdf_samokat_msk['geometry'].apply(lambda x: None if pd.isnull(x) else Point(x.centroid.y, x.centroid.x))

gdf_samokat_msk=get_gdf(df_samokat_msk,'geometry_zone', 4326 )

get_gdf(pd.concat([df_samokat_msk[~pd.isnull(df_samokat_msk['kind'])],gdf_samokat_msk]),'geometry_zone', 4326 ).isnull().mean()

df_samokat_msk.columns

gdf_samokat_fin = get_gdf(pd.concat([df_samokat_msk[~pd.isnull(df_samokat_msk['kind'])],gdf_samokat_msk]),'geometry_zone', 4326 )[['acceptingOrders', 'address_x', 'coverage_id', 'full_response_coverage',
       'full_response_list', 'full_response_settings', 'lavka_ids',
       'minimalAmount', 'showcaseId', 'type', 'utc_dt', 'utc_tm', 'zone','geometry', 'geometry_zone']]

gdf_samokat_fin.rename(columns={'address_x':'address'}, inplace=True)

gdf_samokat_fin['geometry_str']=gdf_samokat_fin['geometry'].astype(str)

gdf_samokat_fin[['acceptingOrders', 'address', 'coverage_id', 
       'minimalAmount', 'showcaseId', 'type', 'utc_dt', 'utc_tm', 'zone','geometry_str', 'geometry_zone']].to_file('gdf_samokat_fin.geojson', driver='GeoJSON')

gdf_samokat_fin[['acceptingOrders', 'address', 'coverage_id', 
       'minimalAmount', 'showcaseId', 'type', 'utc_dt', 'utc_tm', 'zone','geometry_str', 'geometry_zone']].to_csv('gdf_samokat_fin.csv')

gdf_samokat_fin[gdf_samokat_fin['address']=='3-й Хорошёвский 5стр1']

gdf_samokat_mmrkts[gdf_samokat_mmrkts['address']=='3-й Хорошёвский 5стр1']

### Соединяем гексы, дарксторы и зоны самоката и лавки <a class="anchor" id="first-half-bullet"></a>


m1 = folium.Map([55.753637, 37.621179], zoom_start=11, attr='My Data Attribution')
  

transparetn_style = lambda feature: {'fillColor': 'gray','color': 'gray','weight': 0,'fillOpacity':0.00}


layer = folium.GeoJson(
   gdf_samokat_msk[['geometry_zone','address_x']],
    name='zone',
    style_function=lambda feature: {
        'color' : 'black',
        'weight' : 3,
        'fillOpacity' : 0.0,
        },
     tooltip=folium.features.GeoJsonTooltip(
                fields=['address_x',
                       ],
                aliases=['Адрес:',
                      ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
    ).add_to(m1)

layer2 = folium.GeoJson(
    df_lavka.set_geometry('geometry')[['geometry','hex_gid','external_id']],
    name='зоны лавки',
    style_function=lambda feature: {
        'color' : 'feature.properties.external_id',
        'weight' : 3,
        'fillOpacity' : 0.8,
        },
     tooltip=folium.features.GeoJsonTooltip(
                fields=['external_id',
                       ],
                aliases=['Лавка:',
                      ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
    ).add_to(m1)

# layer2 = folium.GeoJson(
#     df_samokat_msk.set_geometry('geometry')[['geometry','address_x']],
#     name='stores Самокат',
#     style_function=lambda feature: {
#         'color' : 'green',
#         'weight' : 3,
#         'fillOpacity' : 0.0,
#         },
#      tooltip=folium.features.GeoJsonTooltip(
#                 fields=['address_x',
#                        ],
#                 aliases=['Самокат:',
#                       ],
#                 style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
#     ).add_to(m1)
m1

df_lavka

gdf_samokat_mmrkts = gdf_samokat_fin[gdf_samokat_fin['type']=='MINIMARKET']

gdf_samokat_mmrkts['acceptingOrders'].value_counts()

 gpd.sjoin(
        gpd.sjoin(
            df_lavka,
            gdf_lavki_loc,
            op='contains',
            how='left'
        ).drop('index_right', axis=1),
        gdf_samokat_fin.set_geometry('geometry'),
        how='left',
        op='contains').isnull().mean()

gdf_lavki_samokats = gpd.sjoin(
    gpd.sjoin(
        get_gdf(
            df_lavka.merge(
            gdf_lavki_loc,
            on=['lavka_id','lavka_slug'],
            how='left',
            suffixes=('_hex','_lavka_store')
        ), 
            'geometry_hex',4326),
        gdf_samokat_mmrkts.set_geometry('geometry')[['address','coverage_id','showcaseId','geometry_str','geometry']],
        how='left',
        op='contains',
        rsuffix='_samokat_store'),
    gdf_samokat_mmrkts[['address','coverage_id','showcaseId','geometry_zone']],
    how='left',
    op='intersects',
    rsuffix='_zone',
    lsuffix ='_store'
)

gdf_lavki_samokats[(gdf_lavki_samokats['coverage_id__zone']!= gdf_lavki_samokats['coverage_id__store'])&(~pd.isnull(gdf_lavki_samokats['coverage_id__store']))]['hex_gid']

gdf_lavki_samokats[gdf_lavki_samokats['hex_gid']=='8911aa710abffff']

gdf_lavki_samokats = gdf_lavki_samokats[(gdf_lavki_samokats['coverage_id__zone']== gdf_lavki_samokats['coverage_id__store'])|(pd.isnull(gdf_lavki_samokats['coverage_id__store']))]

# добавляем geometry_zone
gdf_lavki_samokats = gdf_lavki_samokats.merge(gdf_samokat_mmrkts[['coverage_id','showcaseId','geometry_zone']], left_on=['coverage_id__zone','showcaseId__zone'], 
                        right_on = ['coverage_id','showcaseId'])

gdf_lavki_samokats.isnull().mean()

gdf_lavki_samokats.shape

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['coverage_id__store'])]

gdf_lavki_samokats.to_csv('gdf_lavki_samokats.csv')

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['address__store'])]['C1C2_res_6m'].sum()

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['address__store'])]['C1C2_res_6m'].mean()

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['address__store'])]['C1C2_res_12m'].mean()

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['lavka_slug'])]['C1C2_res_6m'].sum()

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['lavka_slug'])]['C1C2_res_6m'].mean()

gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['lavka_slug'])]['C1C2_res_12m'].mean()

gdf_lavki_samokats[gdf_lavki_samokats['coverage_id']=='53126bf5d50c520805a1df43fe5346ca']['hex_gid'].nunique()

gdf_lavki_samokats[gdf_lavki_samokats['address__zone']=='3-й Хорошёвский 5стр1']['coverage_id__store'].unique()

plt.hist(gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['address__store'])]['C1C2_res_6m'], color='pink', alpha=1, label='samokat')
plt.hist(gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['lavka_slug'])]['C1C2_res_6m'],color='blue', alpha=0.4, label='lavka')
plt.legend()
plt.title('audience coverage with 15cte');

gdf_lavki_samokats.groupby(['external_id','coverage_id__zone']).size()

gdf_lavki_samokats[(~pd.isnull(gdf_lavki_samokats['coverage_id__zone']))&(~pd.isnull(gdf_lavki_samokats['lavka_id']))][['coverage_id__zone','lavka_id','C1C2_res_6m']].drop_duplicates()

gpd_lavka6m = gpd.read_file('lavka_iso6m.geojson')

gpd_lavka6m

import gc

gc.collect()

gpd_lavka_iso=gpd_lavka6m[~pd.isnull(gpd_lavka6m['lavka_id'])]

gpd_lavka_iso.shape

gpd_lavka_iso['lavka_id'].nunique()

#здесь и ниже убираю с1с2 из изохроны не в зоне самоката
gdf_samokat_6m = gpd_lavka6m.merge(gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['coverage_id__store'])][['coverage_id__store','hex_gid']], on='hex_gid')

gdf_lavki_samokats.geometry

samokat_joned = gpd.sjoin(gdf_samokat_6m,gdf_lavki_samokats[['hex_gid','geometry_hex','C1C2_res','coverage_id__zone']], op='intersects', how='inner')

samokat_joned = samokat_joned[samokat_joned['coverage_id__zone']==samokat_joned['coverage_id__store']]

samokat_joned.groupby('coverage_id__store')['C1C2_res'].agg(['sum','size'])

gdf_lavki_samokats['rank']=gdf_lavki_samokats.groupby(['hex_gid'])['C1C2_res'].rank('first')

gdf_lavki_samokats = gdf_lavki_samokats[gdf_lavki_samokats['rank']==1]

df_lavka[['hex_gid','geometry','external_id']]

gpd_lavka_iso =gpd.sjoin(gpd_lavka_iso,df_lavka[['hex_gid','geometry','external_id']], op='intersects', how='inner').drop('index_right', axis=1)

gpd_lavka_iso = gpd_lavka_iso[gpd_lavka_iso['lavka_id']==gpd_lavka_iso['external_id']]

gpd_lavka_iso

gpd_lavka_iso_coverage_inter = gpd_lavka_iso.merge(gdf_lavki_samokats[['hex_gid','coverage_id__zone']],
                   on='hex_gid', how='inner')

gpd_lavka_iso_coverage_inter['hex_gid'].value_counts()

gpd_lavka_iso_coverage_inter = gpd_lavka_iso_coverage_inter.merge(df_lavka[['hex_gid','C1C2_res']])

gpd_lavka_iso_coverage_inter['hex_gid'].value_counts()

gpd_lavka_iso_coverage_inter[gpd_lavka_iso_coverage_inter['coverage_id__zone']=='53126bf5d50c520805a1df43fe5346ca']

gdf_lavki_samokats

gpd_lavka15_samokat_zone_inter = gpd_lavka_iso_coverage_inter.groupby(['coverage_id__zone','lavka_id'])['C1C2_res'].agg(['sum','size']).reset_index().merge(
gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['coverage_id__store'])][['coverage_id__store','C1C2_res_6m','address__zone']],
    left_on = 'coverage_id__zone', right_on = 'coverage_id__store'
).merge(
gdf_lavki_samokats[~pd.isnull(gdf_lavki_samokats['lavka_id'])][['lavka_id','C1C2_res_6m']],
    on = 'lavka_id'
)

gpd_lavka15_samokat_zone_inter.rename(columns={'sum':'C1C2_res_inter_lavka',
                                               'C1C2_res_6m_x':'C1C2_res_15samokat',
                                               'C1C2_res_6m_y':'C1C2_res_15lavka',
                                               'size':'hex_intersected'
                                              }, inplace=True)

gpd_lavka15_samokat_zone_inter[gpd_lavka15_samokat_zone_inter['C1C2_res_15lavka']==gpd_lavka15_samokat_zone_inter['C1C2_res_inter_lavka']]

gpd_lavka15_samokat_zone_inter['coverage_id__zone'].value_counts()

gpd_lavka15_samokat_zone_inter[gpd_lavka15_samokat_zone_inter['address__zone']=='Зарайская 37']

# размеры лавки самоката
gpd_lavka15_samokat_zone_inter = gpd_lavka15_samokat_zone_inter.merge(
gdf_lavki_samokats.groupby('coverage_id__zone')['C1C2_res'].agg(['sum','size']).reset_index(),
    on='coverage_id__zone'
)

gpd_lavka15_samokat_zone_inter.rename(columns={'sum':'C1C2_res_zone_samokat',
                                
                                               'size':'hex_zone_samokat'
                                              }, inplace=True)

gpd_lavka15_samokat_zone_inter

gpd_lavka15_samokat_zone_inter.to_csv('gpd_lavka15_samokat_zone_inter_0106.csv')

gpd_lavka15_samokat_zone_inter.merge(samokat_joned.groupby('coverage_id__store')['C1C2_res'].agg(['sum','size']), how='left', on='coverage_id__store')



samokat_joned.groupby('coverage_id__store')['C1C2_res'].agg(['sum','size']).to_csv('gdf_samokat_c1c2_15.csv')

samokat_orders = pd.read_csv('Table view_crosstab.csv')

samokat_orders.head()

gdf_lavki_samokats.isnull().mean()

gdf_lavki_samokats[gdf_lavki_samokats[]]